# DATASET



## PART 1 : MODIFIED VGG 16

In [1]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

full_model = VGG16(weights='imagenet', input_shape=(None, None, 3), include_top=False)

/home/soumyadeep/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
layers = [ l for l in full_model.layers]

In [3]:
from keras.layers import Input

x = layers[0].output
to_avoid = [3, 6, 10, 19, 20, 21]
for i in range(1, len(layers)):
  if ( i in to_avoid ):
    continue
  layers[i].trainable = False
  x = layers[i](x)

model = Model(inputs = [layers[0].input], outputs = [x])

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)   295168    
_________________________________________________________________
block3_conv2 (Conv2D)        (None, None, None, 256)   590080    
__________

## PART 2 : STORING FEATURE MAPS

In [5]:
import os
import cv2
import numpy as np
import time
import h5py

anchors = [(12,12,2,2),(19,20,3,4),(22,22,5,5),(34,35,6,7),(42,42,8,8),(49,50,9,10)]

dataset_folder = 'DATASET'
feature_folder = 'FEATURE_MAPS'

for im in os.listdir(dataset_folder):
    if 'image_' in im:
        output_name = os.path.join(feature_folder, 'image_' + im.split('.')[0].split('_')[1] + '.h5')
        if os.path.isfile(output_name):
            print(im, 'already done')
            continue
        image = cv2.imread(os.path.join(dataset_folder, im))
        image = image.reshape(1, *image.shape)
        print('Processing', im, image.shape)
        start = time.time()
        result = model.predict(image)
        end = time.time()
        print(result.shape, "Time taken", end-start)
        with h5py.File(output_name, 'w') as f:
            f.create_dataset("default", data=result, compression='gzip')

('image_0032.jpg', 'already done')
('image_0064.jpg', 'already done')
('image_0045.jpg', 'already done')
('image_0008.jpg', 'already done')
('image_0010.jpg', 'already done')
('image_0040.jpg', 'already done')
('image_0061.jpg', 'already done')
('image_0093.jpg', 'already done')
('image_0103.jpg', 'already done')
('image_0017.jpg', 'already done')
('image_0020.jpg', 'already done')
('image_0070.jpg', 'already done')
('image_0079.jpg', 'already done')
('image_0022.jpg', 'already done')
('image_0082.jpg', 'already done')
('image_0125.jpg', 'already done')
('image_0037.jpg', 'already done')
('image_0123.jpg', 'already done')
('image_0025.jpg', 'already done')
('image_0004.jpg', 'already done')
('image_0120.jpg', 'already done')
('image_0094.jpg', 'already done')
('image_0018.jpg', 'already done')
('image_0091.jpg', 'already done')
('image_0099.jpg', 'already done')
('image_0109.jpg', 'already done')
('image_0009.jpg', 'already done')
('image_0111.jpg', 'already done')
('image_0121.jpg', '

## PART 2 : STORING ANCHORS

In [5]:
import os
import h5py 
import numpy as np
from tqdm import tqdm

anchors = [(12,12,2,2),(19,20,3,4),(22,22,5,5),(34,35,6,7),(42,42,8,8),(49,50,9,10)]

feature_folder = 'FEATURE_MAPS'
anchor_folder = 'ANCHORS'

for fm in tqdm(os.listdir(feature_folder)):
    feature_map = h5py.File(feature_folder+"/"+fm,'r+')
    feature_map = np.array(feature_map.get('default'))
    [ n , n_i, n_j, n_k ] = feature_map.shape
    output_name = anchor_folder+'/'+fm.split('.')[0]+'.h5'
    if os.path.isfile(output_name):
        continue
    with h5py.File(output_name,'w') as hf:
        cnt = 0    
        for w_l, w_r, h_l, h_r in anchors:
            for i in range( w_l, n_i-w_r, 10):
                for j in range( h_l, n_j-h_r, 10):                             
                    anchor = feature_map[:,i-w_l:i+w_r,j-h_l:j+h_r,:]                
                    hf.create_dataset('anchor_'+str(cnt), data=anchor, compression="gzip")
                    cnt += 1

100%|██████████| 126/126 [01:01<00:00,  3.40it/s]
